# 2024-07-03 CDC ADRIO Demo

_author: Trevor Johnson_

This devlog will demonstrate the functionality of a new 'CDC ADRIO maker' which fetches data from various CDC and HealthData datasets. Six datasets are currenlty included, each with their own limitations and set of supported attributes. A comprehensive list of attributes and their limitations can be found below.

| Attribute Name | Dates | Granularity | Dataset | Description |
| --- | --- | --- | --- | --- |
| covid_cases_per_100k | 2/24/2022 - 5/4/2023 | County, State | United States COVID 19 Community Levels by County | Weekly number of COVID\-19 cases per 100k population. |
| covid_hospitalizations_per_100k | 2/24/2022 - 5/4/2023 | County, State | United States COVID 19 Community Levels by County | Weekly number of COVID\-19 hospitalizations per 100k population. |
| covid_hospitalization_avg_facility | 12/13/2020 - 5/10/2023 | County, State | COVID-19 Reported Patient Impact and Hospital Capacity by Facility | Weekly averages of COVID\-19 hospitalizations reported by facility. |
| covid_hospitalization_sum_facility | 12/13/2020 - 5/10/2023 | County, State | COVID-19 Reported Patient Impact and Hospital Capacity by Facility | Weekly sums of COVID\-19 hospitalizations reported by facility. |
| influenza_hospitalization_avg_facility | 12/13/2020 - 5/10/2023 | County, State | COVID-19 Reported Patient Impact and Hospital Capacity by Facility | Weekly averages of influenza hospitalizations reported by facility. |
| influenza_hospitalization_sum_facility | 12/13/2020 - 5/10/2023 | County, State | COVID-19 Reported Patient Impact and Hospital Capacity by Facility | Weekly sums of influenza hospitalizations reported by facility. |
| covid_hospitalization_avg_state | 1/04/2020 - present | State | Weekly United States Hospitalization Metrics by Jurisdiction | Weekly averages of COVID\-19 hospitalizations reported by state. |
| covid_hospitalization_sum_state | 1/04/2020 - present | State | Weekly United States Hospitalization Metrics by Jurisdiction | Weekly sums of COVID\-19 hospitalizations reported by state. |
| influenza_hospitalization_avg_state | 1/04/2020 - present | State | Weekly United States Hospitalization Metrics by Jurisdiction | Weekly averages of influenza hospitalizations reported by state. |
| influenza_hospitalization_avg_state | 1/04/2020 - present | State | Weekly United States Hospitalization Metrics by Jurisdiction | Weekly sums of influenza hospitalizations reported by state. |
| full_covid_vaccinations | 12/13/2020 - 5/10/2024 | County, State | COVID-19 Vaccinations in the United States,County | Weekly cumulative total of individuals who have compeleted a series of COVID\-19 vaccinations. |
| one_dose_covid_vaccinations | 12/13/2020 - 5/10/2024 | County, State | COVID-19 Vaccinations in the United States,County | Weely cumulative total of individuals who have recieved at least one dose of COVID\-19 vaccination. |
| covid_booster_doses | 12/13/2020 - 5/10/2024 | County, State | COVID-19 Vaccinations in the United States,County | Weekly cumulative total of COVID\-19 booster doses administered. |
| covid_deaths_county | 1/4/2020 - 4/5/2024 | County, State | AH COVID-19 Death Counts by County and Week, 2020-present | Weekly number of COVID\-19 deaths reported by county.  |
| covid_deaths_state | 1/4/2020 - present | State | Provisional COVID-19 Death Counts by Week Ending Date and State | Weekly number of COVID\-19 deaths reported by state. |
| influenza_deaths | 1/4/2020 - present | State | Provisional COVID-19 Death Counts by Week Ending Date and State | Weekly number of influenza deaths reported by state. |

## **Demo**
ADRIO maker functionality will be demonstrated one dataset at a time. A brief description of each dataset will be given and one ADRIO for each of its attributes will be created and run.

In [1]:
from epymorph.geography.us_census import StateScope
from epymorph.time import TimeFrame

# We'll use these two scopes throughout...
state_scope = StateScope.in_states(["AZ", "CO"], year=2022)
county_scope = state_scope.lower_granularity()

### **United States COVID 19 Community Levels by County**
This dataset is used to fetch data on reported COVID-19 cases and hospitalizations per 100k population.

- Supported attributes: covid_cases_per_100k, covid_hospitalizations_per_100k
- Available date range: 2/24/2022 to 5/4/2023
- Granularity: county, state

https://healthdata.gov/dataset/United-States-COVID-19-Community-Levels-by-County/nn5b-j5u9/about_data

In [2]:
from epymorph.adrio.cdc import CovidCasesPer100k, CovidHospitalizationsPer100k

time = TimeFrame.range("2022-02-24", "2022-03-10")
context = {"scope": state_scope, "time_frame": time}

cases = CovidCasesPer100k().with_context(**context).evaluate()
print(f"COVID cases per 100k:\n{cases}\n")

hospitalizations = CovidHospitalizationsPer100k().with_context(**context).evaluate()
print(f"COVID hospitalizations per 100k:\n{hospitalizations}")

COVID cases per 100k:
[[('2022-02-24',  3217.93) ('2022-02-24', 12475.51)]
 [('2022-03-03',  1018.81) ('2022-03-03',  8795.74)]
 [('2022-03-10',  1912.86) ('2022-03-10', 12280.08)]]

COVID hospitalizations per 100k:
[[('2022-02-24', 334.7) ('2022-02-24', 822.5)]
 [('2022-03-03', 232.6) ('2022-03-03', 445.5)]
 [('2022-03-10', 186. ) ('2022-03-10', 332.5)]]


### **COVID-19 Reported Patient Impact and Hospital Capacity by Facility**
This dataset is used to fetch hospitalization data for COVID-19 and other respiratory illnesses.

- Supported attributes: covid_hospitalization_avg_facility, covid_hospitalization_sum_facility, influenza_hospitalization_avg_facility, influenza_hospitalization_sum_facility
- Available date range: 12/13/2020 to 5/10/2023
- Granularity: county, state

https://healthdata.gov/Hospital/COVID-19-Reported-Patient-Impact-and-Hospital-Capa/anag-cw7u/about_data


In [3]:
from epymorph.adrio.cdc import (
    CovidHospitalizationAvgFacility,
    CovidHospitalizationSumFacility,
    InfluenzaHospitalizationSumFacility,
    InfluenzaHosptializationAvgFacility,
)


# COVID

time = TimeFrame.range("2020-12-13", "2021-01-10")
context = {"scope": county_scope, "time_frame": time}

# these attributes contain many -999,999 "sentinel values" to suppress values < 4
# an additional integer parameter 0-3 is required to specify what to replace these with
covid_avg = CovidHospitalizationAvgFacility(0).with_context(**context).evaluate()
print(f"COVID hospitalization average:\n{covid_avg[:, 0:3]}\n")

covid_sum = CovidHospitalizationSumFacility(0).with_context(**context).evaluate()
print(f"COVID hospitalization sum:\n{covid_sum[:, 0:3]}\n")


# Flu

time = TimeFrame.range("2022-11-13", "2022-12-11")
context = {"scope": county_scope, "time_frame": time}

flu_avg = InfluenzaHosptializationAvgFacility(0).with_context(**context).evaluate()
print(f"Influenza hospitalization average:\n{flu_avg[:, 0:3]}\n")

flu_sum = InfluenzaHospitalizationSumFacility(0).with_context(**context).evaluate()
print(f"Influenza hospitalization sum:\n{flu_sum[:, 0:3]}")

/home/tcoles/Workspaces/Epymorph/epymorph/adrio/cdc.py:89: UserWarning: 173 values < 4 were replaced with 0 in returned data.
  warn(


COVID hospitalization average:
[[('2020-12-13',  6.4  ) ('2020-12-13',  7.425) ('2020-12-13', 23.3  )]
 [('2020-12-20',  6.4  ) ('2020-12-20',  6.9  ) ('2020-12-20', 26.85 )]
 [('2020-12-27',  5.7  ) ('2020-12-27',  7.275) ('2020-12-27', 25.85 )]
 [('2021-01-03',  5.3  ) ('2021-01-03',  8.575) ('2021-01-03', 33.65 )]
 [('2021-01-10',  5.9  ) ('2021-01-10',  8.025) ('2021-01-10', 30.2  )]]



/home/tcoles/Workspaces/Epymorph/epymorph/adrio/cdc.py:89: UserWarning: 38 values < 4 were replaced with 0 in returned data.
  warn(


COVID hospitalization sum:
[[('2020-12-13',  45.  ) ('2020-12-13',  59.  ) ('2020-12-13', 176.  )]
 [('2020-12-20',  45.  ) ('2020-12-20',  56.25) ('2020-12-20', 188.  )]
 [('2020-12-27',  40.  ) ('2020-12-27',  59.5 ) ('2020-12-27', 194.5 )]
 [('2021-01-03',  37.  ) ('2021-01-03',  66.25) ('2021-01-03', 235.5 )]
 [('2021-01-10',  41.  ) ('2021-01-10',  60.25) ('2021-01-10', 222.  )]]



/home/tcoles/Workspaces/Epymorph/epymorph/adrio/cdc.py:89: UserWarning: 308 values < 4 were replaced with 0 in returned data.
  warn(


Influenza hospitalization average:
[[('2022-11-13', 0.  ) ('2022-11-13', 0.  ) ('2022-11-13', 3.55)]
 [('2022-11-20', 0.  ) ('2022-11-20', 0.  ) ('2022-11-20', 4.35)]
 [('2022-11-27', 0.  ) ('2022-11-27', 0.  ) ('2022-11-27', 5.7 )]
 [('2022-12-04', 0.  ) ('2022-12-04', 0.  ) ('2022-12-04', 3.85)]
 [('2022-12-11', 0.  ) ('2022-12-11', 0.  ) ('2022-12-11', 3.5 )]]

Influenza hospitalization sum:
[[('2022-11-13',  0.  ) ('2022-11-13',  0.  ) ('2022-11-13', 29.5 )]
 [('2022-11-20',  0.  ) ('2022-11-20',  1.  ) ('2022-11-20', 30.5 )]
 [('2022-11-27', 10.  ) ('2022-11-27',  2.25) ('2022-11-27', 40.  )]
 [('2022-12-04',  5.  ) ('2022-12-04',  1.25) ('2022-12-04', 27.  )]
 [('2022-12-11',  8.  ) ('2022-12-11',  0.  ) ('2022-12-11', 24.5 )]]


/home/tcoles/Workspaces/Epymorph/epymorph/adrio/cdc.py:89: UserWarning: 95 values < 4 were replaced with 0 in returned data.
  warn(


### **Weekly United States Hospitalization Metrics by Jurisdiction**
Like the previous dataset, this dataset is used to fetch hospitalization data for COVID-19 and other respiratory illnesses. Unlike the previous dataset however, it includes metrics reported voluntarily after the end of the manditory reporting period and is limited to state granularity.

- Supported attributes: covid_hospitalization_avg_state, covid_hospitalization_sum_state, influenza_hospitalization_avg_state, influenza_hospitalization_sum_state
- Available date range: 1/04/2020 to present. Data reported voluntary past 5/1/2024.
- Granularity: state

https://data.cdc.gov/Public-Health-Surveillance/Weekly-United-States-Hospitalization-Metrics-by-Ju/aemt-mg7g/about_data

In [4]:
from epymorph.adrio.cdc import (
    CovidHospitalizationAvgState,
    CovidHospitalizationSumState,
    InfluenzaHospitalizationAvgState,
    InfluenzaHospitalizationSumState,
)


# COVID

time = TimeFrame.range("2020-12-19", "2021-01-16")
context = {"scope": state_scope, "time_frame": time}

covid_avg = CovidHospitalizationAvgState().with_context(**context).evaluate()
print(f"COVID hospitalization average:\n{covid_avg}\n")

covid_sum = CovidHospitalizationSumState().with_context(**context).evaluate()
print(f"COVID hospitalization sum:\n{covid_sum}\n")


# Flu

flu_avg = InfluenzaHospitalizationAvgState().with_context(**context).evaluate()
flu_sum = InfluenzaHospitalizationSumState().with_context(**context).evaluate()

print(f"Influenza hospitalization average:\n{flu_avg}\n")
print(f"Influenza hospitalization sum:\n{flu_sum}\n")

COVID hospitalization average:
[[('2020-12-19', 452.  ) ('2020-12-19', 199.43)]
 [('2020-12-26', 472.43) ('2020-12-26', 164.  )]
 [('2021-01-02', 495.  ) ('2021-01-02', 150.14)]
 [('2021-01-09', 567.14) ('2021-01-09', 136.29)]
 [('2021-01-16', 536.57) ('2021-01-16', 129.57)]]

COVID hospitalization sum:
[[('2020-12-19', 3164.) ('2020-12-19', 1396.)]
 [('2020-12-26', 3307.) ('2020-12-26', 1148.)]
 [('2021-01-02', 3465.) ('2021-01-02', 1051.)]
 [('2021-01-09', 3970.) ('2021-01-09',  954.)]
 [('2021-01-16', 3756.) ('2021-01-16',  907.)]]

Influenza hospitalization average:
[[('2020-12-19', 1.57) ('2020-12-19', 0.71)]
 [('2020-12-26', 1.14) ('2020-12-26', 0.71)]
 [('2021-01-02', 6.29) ('2021-01-02', 0.  )]
 [('2021-01-09', 1.43) ('2021-01-09', 0.  )]
 [('2021-01-16', 2.  ) ('2021-01-16', 0.57)]]

Influenza hospitalization sum:
[[('2020-12-19', 11.) ('2020-12-19',  5.)]
 [('2020-12-26',  8.) ('2020-12-26',  5.)]
 [('2021-01-02', 44.) ('2021-01-02',  0.)]
 [('2021-01-09', 10.) ('2021-01-09',

### **COVID-19 Vaccinations in the United States,County**
This dataset is used to fetch cumulative COVID-19 vaccination data.

- Supported attributes: full_covid_vaccinations, one_dose_covid_vaccinations, covid_booster_doses
- Available date range: 12/13/2020 to 5/10/2024.
- Granularity: county, state

https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh/about_data

In [5]:
from epymorph.adrio.cdc import (
    CovidBoosterDoses,
    FullCovidVaccinations,
    OneDoseCovidVaccinations,
)

time = TimeFrame.range("2021-12-13", "2021-12-19")
context = {"scope": state_scope, "time_frame": time}

full = FullCovidVaccinations().with_context(**context).evaluate()
print(f"Full COVID vaccinations:\n{full}\n")

one = OneDoseCovidVaccinations().with_context(**context).evaluate()
print(f"One dose COVID vaccinations:\n{one}\n")

booster = CovidBoosterDoses().with_context(**context).evaluate()
print(f"COVID booster doses:\n{booster}")

Full COVID vaccinations:
[[('2021-12-13', 3982347.) ('2021-12-13', 3641750.)]
 [('2021-12-14', 3984751.) ('2021-12-14', 3647478.)]
 [('2021-12-15', 3991116.) ('2021-12-15', 3653514.)]
 [('2021-12-16', 3997264.) ('2021-12-16', 3658734.)]
 [('2021-12-17', 4002606.) ('2021-12-17', 3663387.)]
 [('2021-12-18', 4008088.) ('2021-12-18', 3666965.)]
 [('2021-12-19', 4013034.) ('2021-12-19', 3671380.)]]

One dose COVID vaccinations:
[[('2021-12-13', 4670074.) ('2021-12-13', 4003978.)]
 [('2021-12-14', 4672924.) ('2021-12-14', 4007551.)]
 [('2021-12-15', 4679809.) ('2021-12-15', 4012785.)]
 [('2021-12-16', 4687174.) ('2021-12-16', 4018114.)]
 [('2021-12-17', 4693615.) ('2021-12-17', 4023444.)]
 [('2021-12-18', 4701618.) ('2021-12-18', 4028479.)]
 [('2021-12-19', 4709211.) ('2021-12-19', 4033860.)]]

COVID booster doses:
[[('2021-12-13',       0.) ('2021-12-13',       0.)]
 [('2021-12-14',       0.) ('2021-12-14',       0.)]
 [('2021-12-15', 1007483.) ('2021-12-15', 1284522.)]
 [('2021-12-16', 102

### **AH COVID-19 Death Counts by County and Week, 2020-present**
This dataset is used to fetch data on COVID-19 deaths.

- Supported attributes: covid_deaths_county
- Available date range: 1/4/2020 to 4/5/2024.
- Granularity: county, state

https://data.cdc.gov/NCHS/AH-COVID-19-Death-Counts-by-County-and-Week-2020-p/ite7-j2w7/about_data

In [6]:
from epymorph.adrio.cdc import CovidDeathsCounty

time = TimeFrame.range("2021-01-04", "2021-02-06")
context = {"scope": state_scope, "time_frame": time}

deaths = CovidDeathsCounty().with_context(**context).evaluate()
print(f"COVID deaths:\n{deaths}\n")

COVID deaths:
[[('2021-01-09', 102.33333333) ('2021-01-09',   3.6       )]
 [('2021-01-16', 137.14285714) ('2021-01-16',   2.65957447)]
 [('2021-01-23', 115.75      ) ('2021-01-23',   2.06382979)]
 [('2021-01-30',  76.5       ) ('2021-01-30',   1.84      )]
 [('2021-02-06',  79.25      ) ('2021-02-06',   1.31914894)]]



### **Provisional COVID-19 Death Counts by Week Ending Date and State**
This dataset is used to fetch data on COVID-19 and influenza deaths. It is continuously updated but only available for state granularity.

- Supported attributes: covid_deaths_state, influenza_deaths
- Available date range: 1/4/2020 to present.
- Granularity: state

https://data.cdc.gov/NCHS/Provisional-COVID-19-Death-Counts-by-Week-Ending-D/r8kw-7aab/about_data

In [7]:
from epymorph.adrio.cdc import CovidDeathsState, InfluenzaDeathsState

# COVID

time = TimeFrame.range("2021-01-04", "2021-01-31")
context = {"scope": state_scope, "time_frame": time}

covid_deaths = CovidDeathsState().with_context(**context).evaluate()
print(f"COVID deaths:\n{covid_deaths}\n")


# Flu

time = TimeFrame.range("2022-11-26", "2022-12-17")
context = {"scope": state_scope, "time_frame": time}

flu_deaths = InfluenzaDeathsState().with_context(**context).evaluate()
print(f"Influenza deaths:\n{flu_deaths}")

COVID deaths:
[[('2021-01-09',  942.) ('2021-01-09',  211.)]
 [('2021-01-16',  996.) ('2021-01-16',  165.)]
 [('2021-01-23',  959.) ('2021-01-23',  158.)]
 [('2021-01-30',  776.) ('2021-01-30',  141.)]
 [('2021-01-31', 4032.) ('2021-01-31',  754.)]]

Influenza deaths:
[[('2022-11-26', nan) ('2022-11-26', 10.)]
 [('2022-11-30', 27.) ('2022-11-30', 30.)]
 [('2022-12-03', 11.) ('2022-12-03', 13.)]
 [('2022-12-10', 18.) ('2022-12-10', 15.)]
 [('2022-12-17', nan) ('2022-12-17', 10.)]]
